## Description of the Project


### >>> Data Analysis - Predictive Models

* House price is always the biggest consideration for either the seller or the buyer. The **goal** of this project is to help the real estate company to **1) accurately predict the future house price** by using the components that the agents collect from the customers and the old sales data they already have in the past few years. **2) recommend suitable house types** that better matches their customers' needs.  

* I find a dataset online and attach it in my folder. In this project, I will do the **data analysis** first and build two **predictive models** based on the data I have. This model can not only be used for this dataset, as long as changing some variables, it can apply to other datasets with the similar purpose.

## Project Code

### Step 1: import the libraries including numpy, pandas and sklearn

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression  
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

### Step 2: import the file

In [2]:
df = pd.read_excel(r'data.xlsx')

### Step 3: take a look at the data

In [3]:
df.sample(5)

id                   Community  CommunityNumber  CommunityType  \
506    888              Velvendo - 70s      41040650000  First Move Up   
881   1944         Pima Crossing - 50s      41050400000     Entry Plus   
1356   875  Sedella 4B.2 - 75s - Cadiz      41040330000  First Move Up   
554   2945        Zanjero Trails - 58s      41051450000      LiVE.NOW.   
1571  2029              Florenza - 75s      41050850000  First Move Up   

     SaleNumber   SaleDate  CloseDate  BasePrice  TotalApprovedSalesPrice  \
506    S-099489 2015-12-22 2016-01-29          0                   392714   
881    S-121167 2017-02-28 2017-03-29          0                   222337   
1356   S-099029 2015-12-31 2016-09-09          0                   365449   
554    S-140585 2018-02-17 2018-04-03          0                   305000   
1571   S-122507 2017-04-30 2017-10-10          0                   421164   

      LotNumber  ... DerivedPriceFlag LoanAmt                    Lender  \
506          15  ...              NaN  401250        CALIBER HOME LOANS   
881          88  ...              NaN       0      ( CASH TRANSACTION )   
1356        102  ...              NaN       0      ( CASH TRANSACTION )   
554          29  ...              NaN       0      ( CASH TRANSACTION )   
1571        115  ...              NaN  230400  CARDINAL FINANCIAL CO LP   

                 TitleCompany  DocumentNumber  YrBuilt Stories.1 Bedrooms.1  \
506   GRAND CANYON TITLE AGCY          497773     2015         1        NaN   
881                       NaN          416624     2001         1        NaN   
1356                      NaN          245893     2006         2        NaN   
554       EQUITY TITLE AGENCY          469765     1999         1        NaN   
1571          PRINCIPAL TITLE          305528     2008         2        NaN   

     Bathrooms.1         Submarket  
506          NaN       East Valley  
881          NaN       East Valley  
1356         NaN  Southwest Valley  
554          NaN  Northwest Valley  
1571         NaN     Phoenix Metro  

[5 rows x 65 columns]

In [4]:
df.columns

Index(['id', 'Community', 'CommunityNumber', 'CommunityType', 'SaleNumber',
       'SaleDate', 'CloseDate', 'BasePrice', 'TotalApprovedSalesPrice',
       'LotNumber', 'LotAddress', 'LotCity', 'LotPostalCode', 'LotState',
       'CommunityLatitude', 'CommunityLongitude', 'CustomerFirstName',
       'CustomerLastName', 'CustomerPrimaryAddress', 'CustomerPrimaryCity',
       'CustomerPrimaryZipCode', 'CustomerPrimary State',
       'CobuyerContractFirstName', 'CobuyerContractLastName',
       'CobuyerAddressLine1', 'CobuyerCity', 'CobuyerState', 'CobuyerZip',
       'MLSNumber', 'PlanNumber', 'SquareFeet', 'Bedrooms', 'Bathrooms',
       'Stories', 'GarageCapacity', 'id.1', 'Parcel', 'County',
       'LandUseHousingType', 'ZoningCode', 'LotSF', 'FinSF',
       'FinSF-Assessor-Based', 'DerivedSqFt Flag', 'Owner', 'SiteAddress',
       'SiteCity', 'SiteZip', 'SubDivision', 'TranType', 'TranDesc', 'Seller',
       'Purchaser', 'SaleDate.1', 'SalePrice', 'DerivedPriceFlag', 'LoanAmt',
      

### Step 4: data preprocessing -1

In [5]:
#create new column - house price difference

df['price_diff'] = df['TotalApprovedSalesPrice'] - df['SalePrice']

In [6]:
#drop the columns that are unrelated

df.drop(['id', 'Community', 'CommunityNumber', 'CommunityType', 'SaleNumber',
         'SaleDate', 'CloseDate', 'BasePrice', 'TotalApprovedSalesPrice',
         'LotNumber', 'LotAddress', 'LotCity', 'LotPostalCode', 'LotState',
         'CommunityLatitude', 'CommunityLongitude', 'CustomerFirstName',
         'CustomerLastName', 'CustomerPrimaryAddress', 'CustomerPrimaryCity',
         'CustomerPrimaryZipCode', 'CustomerPrimary State', 'Parcel', 'ZoningCode',
         'CobuyerContractFirstName', 'CobuyerContractLastName', 'Submarket',
         'CobuyerAddressLine1', 'CobuyerCity', 'CobuyerState', 'CobuyerZip',
         'MLSNumber', 'PlanNumber', 'SquareFeet', 'Bedrooms', 'Bathrooms',
         'Stories', 'GarageCapacity', 'Owner','Seller','TitleCompany','TranType',
         'Lender', 'Purchaser', 'SaleDate.1', 'DocumentNumber',
         'FinSF-Assessor-Based', 'DerivedSqFt Flag', 'SiteAddress'],axis=1,inplace=True)

In [7]:
df.shape

(1750, 17)

In [8]:
#check the missing values

df.isna().sum()

id.1                     0
County                   0
LandUseHousingType       0
LotSF                    0
FinSF                    0
SiteCity                 0
SiteZip                  0
SubDivision              0
TranDesc                 0
SalePrice                0
DerivedPriceFlag      1750
LoanAmt                  0
YrBuilt                  0
Stories.1                0
Bedrooms.1            1705
Bathrooms.1           1705
price_diff               0
dtype: int64

In [9]:
#define threshold for dropping columns that have less than thresh number of non_Nans
percent = int(0.7*(df.shape[0]))
df.dropna(thresh=percent, axis=1)

id.1     County   LandUseHousingType  LotSF  FinSF         SiteCity  \
0      23420   Maricopa  Det | Single Family   5300   1618          PHOENIX   
1      27307   Maricopa  Det | Single Family   7934   1443          PHOENIX   
2      31004   Maricopa  Det | Single Family   7611   1958          PHOENIX   
3      55693   Maricopa  Det | Single Family   3576   1903          PHOENIX   
4      60926   Maricopa  Det | Single Family   4294   2365          PHOENIX   
...      ...        ...                  ...    ...    ...              ...   
1745  567755      Pinal  Det | Single Family   7330   1770  APACHE JUNCTION   
1746  567782   Maricopa  Det | Single Family  12684   2544          PHOENIX   
1747  567791  San Diego  Det | Single Family   6538    814       SAN MARCOS   
1748  567824   Maricopa  Det | Single Family  14170   2681             MESA   
1749  567822       Pima  Det | Single Family   4726   1654           TUCSON   

      SiteZip               SubDivision        TranDesc  SalePrice  LoanAmt  \
0       85024              EAGLE SUMMIT  Regular Resale     295000   265000   
1       85027           FOOTHILLS NORTH  Regular Resale     216499   216499   
2       85029                  FAIRWOOD  Regular Resale     182000   172900   
3       85037    CAMELBACK RANCH VILLAS  Regular Resale     167000   163690   
4       85042             BLOSSOM HILLS  Regular Resale     269000   242100   
...       ...                       ...             ...        ...      ...   
1745    85120              ARROYO VERDE  Regular Resale     245000   222950   
1746    85028  PHOENIX MOUNTAIN ESTATES  Regular Resale     400888   320710   
1747    92069          SAN MARCOS MANOR  Regular Resale     365000   358388   
1748    85203          PLEASANT SHADOWS  Regular Resale     358000        0   
1749    85746      CAMBRIE MIDVALE PARK  Regular Resale     195000   191468   

      YrBuilt  Stories.1  price_diff  
0        1998          1      117214  
1        1996          1      148337  
2        1973          1       93949  
3        2008          2      266405  
4        2010          2       53556  
...       ...        ...         ...  
1745     1999          1       21950  
1746     1971          1      -82368  
1747        0          0     -132916  
1748     2000          1       86000  
1749     2000          1       68520  

[1750 rows x 14 columns]

### Step 5: data preprocessing -2

In [10]:
#separate the target variable and other features

X = df.drop(["price_diff"],axis=1)
y = df["price_diff"]

In [11]:
#split the train data and test data - ratio 0.8:0.2

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=1)

#ensure to work with a copy of the data - deal with SetttingWithCopyWarning
Xtrain = Xtrain.copy()
Xtest = Xtest.copy()
ytrain = ytrain.copy()
ytest = ytest.copy()

In [12]:
categorical_features = ['County', 'SiteCity', 'SiteZip', 'TranDesc', 'YrBuilt']
numeric_features=[ 'LotSF', 'FinSF', 'SalePrice', 'Stories.1']

In [13]:
#do one-hot encoding on categorical features

categorical_transformer = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore'))])

In [14]:
#normalize the numeric features

numeric_transformer = Pipeline(steps=[('std', StandardScaler())])

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='drop') # remainder='passthrough')

### Step 6: build the model - using Linear Regressor and Random Forest Regressor

In [16]:
#pipeline for linear regression model with PCA

lr = Pipeline(steps=[('pp', preprocessor),("pca",PCA(n_components=8)),
                      ('lr', LinearRegression())])

In [17]:
#fit the model on train data

lr.fit(Xtrain,ytrain)

Pipeline(memory=None,
         steps=[('pp',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('std',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['LotSF', 'FinSF',
                                                   'SalePrice', 'Stories.1']),
                                                 ('cat

In [18]:
#use the model to predict test data

ypred_lr = lr.predict(Xtest)

In [19]:
#see the accuracy score - R-square the smaller the better

def LrScore():
    l = metrics.r2_score(ytest, ypred_lr)
    print("Linear Resgression:", l)

LrScore()    

Linear Resgression: 0.3010621440438789


In [20]:
#pipeline for random forest model with PCA

rf = Pipeline(steps=[('pp', preprocessor),("pca",PCA(n_components=8)),
                      ('rf', RandomForestRegressor(n_estimators=10))])

In [21]:
#fit the model on train data

rf.fit(Xtrain,ytrain)

Pipeline(memory=None,
         steps=[('pp',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('std',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['LotSF', 'FinSF',
                                                   'SalePrice', 'Stories.1']),
                                                 ('cat

In [22]:
#use the model to predict test data

ypred_rf = rf.predict(Xtest)

In [23]:
#see the accuracy score - R-square the smaller the better

def RfScore():
    r = metrics.r2_score(ytest, ypred_rf)
    print("Random Forest:", r)

RfScore()

Random Forest: 0.2774526704710839


### Step 7: find the best model

In [24]:
def FindModel():
    l = metrics.r2_score(ytest, ypred_lr)
    r = metrics.r2_score(ytest, ypred_rf)
    if l < r:
        print('\nRecommend Linear Regression as the model')
    else:
        print('\nRecommend Random Forest as the model')

FindModel()


Recommend Random Forest as the model


## Conclusion

* Compare two models, in this case, the random forest model works better for the prediction. It gives us a lower R-square, meaning the house price is closer to accurate number. So I can recommend this to the real estate company who needs this service. (Note that the score changes, so it depends)